# Capstone Two: Modeling

In [1]:
#os.getcwd()
#os.chdir ('Springboard_Debisree/predicting-cab-booking-cancellations/')

import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime


#ignore warning messages to ensure clean outputs
import warnings
warnings.filterwarnings('ignore')

In [2]:
realtor_data = pd.read_csv('realtor-data.csv')
# drop rows with missing value in the city,price and zip_colde columns

realtor_data2_drop=realtor_data
realtor_data2_drop.dropna(subset =['zip_code','price','city'],inplace=True)
missing = pd.concat([realtor_data2_drop.isnull().sum(), 100 * realtor_data2_drop.isnull().mean()], axis=1)
missing.columns=['count','%']
missing.sort_values(by='%')

,count,%
status,0,0.000000
city,0,0.000000
state,0,0.000000
zip_code,0,0.000000
price,0,0.000000
bath,193804,13.838932
bed,216158,15.435160
acre_lot,357319,25.515022
house_size,449764,32.116227
prev_sold_date,685717,48.964886


In [3]:
# Check data types of columns
for column in realtor_data2_drop.columns[realtor_data2_drop.isnull().sum() > 0]:
    if realtor_data2_drop[column].dtype != 'object':  # Check if column is numeric
        mean_val = realtor_data2_drop[column].mean()
        realtor_data2_drop[column].fillna(mean_val, inplace=True)

In [4]:
# fill missing value with mean
  
for column in realtor_data2_drop.columns[realtor_data2_drop.isnull().sum() > 0]:
    if realtor_data2_drop[column].dtype != 'object':  # Check if column is numeric
        mean_val = realtor_data2_drop[column].mean()
        realtor_data2_drop[column].fillna(mean_val, inplace=True)

In [5]:
# turns status column into a dummy variable 
dummy=pd.get_dummies(realtor_data2_drop['status'])
df=pd.concat([realtor_data2_drop,dummy],axis=1)
df=realtor_data2_drop.merge(dummy,left_index=True,right_index=True)
df.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,for_sale,ready_to_build
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.000000,NaN,105000.0,1,0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.000000,NaN,80000.0,1,0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.000000,NaN,67000.0,1,0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.000000,NaN,145000.0,1,0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,2178.735694,NaN,65000.0,1,0


In [6]:
df=df.drop(columns=['city','state','prev_sold_date','status'])
df.head()

,bed,bath,acre_lot,zip_code,house_size,price,for_sale,ready_to_build
0,3.0,2.0,0.12,601.0,920.000000,105000.0,1,0
1,4.0,2.0,0.08,601.0,1527.000000,80000.0,1,0
2,2.0,1.0,0.15,795.0,748.000000,67000.0,1,0
3,4.0,2.0,0.10,731.0,1800.000000,145000.0,1,0
4,6.0,2.0,0.05,680.0,2178.735694,65000.0,1,0


# Random Forest Model

In [7]:
# Split data into training and testing sets

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='price'), 
                                                    df.price, test_size=0.3, 
                                                    random_state=47)

# Standardize numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)


# Build Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # Example parameters, adjust as needed

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 1064910273911.2472
R-squared: 0.7972877525266895
